# IO

In [ ]:
#| default_exp io

In [ ]:
#| export
import numpy as np
import h5py
import json
import time
import os
from pprint import pprint

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

def write4_data_hdf5(dataD, outF, metaD=None, verb=1):
    """
    Write a dictionary of Python objects and NumPy arrays to an HDF5 file.
    
    Args:
        dataD (dict): Dictionary of data to store.
        outF (str): Output HDF5 filename.
        metaD (dict, optional): Metadata dictionary to store as JSON.
        verb (int): Verbosity level.
    """
    assert dataD is not None
    assert len(outF) > 0
    
    if metaD is not None:
        metaJ = json.dumps(metaD, default=str)
        dataD['meta.JSON'] = metaJ
    
    dtvs = h5py.special_dtype(vlen=str)
    h5f = h5py.File(outF, 'w')
    if verb > 0:
        print('saving data as hdf5:', outF)
        start = time.time()
    
    for item in dataD:
        rec = dataD[item]
        if verb > 1: 
            print('x=', item, type(rec))
        if isinstance(rec, str):  # special case
            dset = h5f.create_dataset(item, (1,), dtype=dtvs)
            dset[0] = rec
            if verb > 0:
                print('h5-write :', item, 'as string', dset.shape, dset.dtype)
            continue
        if not isinstance(rec, np.ndarray):  # packs a single value into np-array
            rec = np.array([rec])
        h5f.create_dataset(item, data=rec)
        if verb > 0:
            print('h5-write :', item, rec.shape, rec.dtype)
    
    h5f.close()
    xx = os.path.getsize(outF) / 1048576
    print('closed  hdf5:', outF, ' size=%.2f MB, elaT=%.1f sec' % (xx, (time.time() - start)))

def read4_data_hdf5(inpF, verb=1):
    """
    Read data and metadata from an HDF5 file.
    
    Args:
        inpF (str): Input HDF5 filename.
        verb (int): Verbosity level.
    
    Returns:
        tuple: (data dictionary, metadata dictionary or None)
    """
    if verb > 0:
        print('read data from hdf5:', inpF)
        start = time.time()
    try:
        with h5py.File(inpF, 'r') as h5f:
            objD = {}
            for x in h5f.keys():
                if verb > 1:
                    print('\nitem=', x, type(h5f[x]), h5f[x].shape, h5f[x].dtype)
                if h5f[x].dtype == object:
                    obj = h5f[x][:]
                    if verb > 0:
                        print('read str:', x, len(obj), type(obj))
                else:
                    obj = h5f[x][:]
                    if verb > 0:
                        print('read obj:', x, obj.shape, obj.dtype)
                objD[x] = obj
            try:
                inpMD = json.loads(objD.pop('meta.JSON')[0])
                if verb > 1:
                    print('  recovered meta-data with %d keys' % len(inpMD))
            except:
                inpMD = None
            if verb > 0:
                print(' done h5, num rec:%d  elaT=%.1f sec' % (len(objD), (time.time() - start)))
            return objD, inpMD
    except OSError as e:
        print(f"Error: {e}")
